In [1]:
import pandas as pd
import os
from sklearn import linear_model
from sklearn.metrics import accuracy_score
import numpy as np

current = os.getcwd() 
train_file = pd.read_csv(current + r"\..\rawdata\train.csv")
test_file = pd.read_csv(current  + r"\..\rawdata\test.csv")

# file = pd.concat([train_file, test_file])
# file.head()
train_file1 = train_file.loc[:, ~train_file.columns.isin(['User_ID', 'Product_ID'])]
test_file1 = test_file.loc[:, ~test_file.columns.isin(['User_ID', 'Product_ID'])]
checkpoint = train_file.shape[0]
# print(checkpoint)
train_file.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [2]:
file = pd.concat([train_file1, test_file1])
file

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,F,0-17,10,A,2,0,3,NaN,NaN,8370.0
1,F,0-17,10,A,2,0,1,6.0,14.0,15200.0
2,F,0-17,10,A,2,0,12,NaN,NaN,1422.0
3,F,0-17,10,A,2,0,12,14.0,NaN,1057.0
4,M,55+,16,C,4+,0,8,NaN,NaN,7969.0
...,...,...,...,...,...,...,...,...,...,...
233594,F,26-35,15,B,4+,1,8,NaN,NaN,NaN
233595,F,26-35,15,B,4+,1,5,8.0,NaN,NaN
233596,F,26-35,15,B,4+,1,1,5.0,12.0,NaN
233597,F,46-50,1,C,4+,0,10,16.0,NaN,NaN


In [3]:
file['Product_Category_2'].fillna(0, inplace=True)
file['Product_Category_3'].fillna(0, inplace=True)
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 233598
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      783667 non-null  object 
 1   Age                         783667 non-null  object 
 2   Occupation                  783667 non-null  int64  
 3   City_Category               783667 non-null  object 
 4   Stay_In_Current_City_Years  783667 non-null  object 
 5   Marital_Status              783667 non-null  int64  
 6   Product_Category_1          783667 non-null  int64  
 7   Product_Category_2          783667 non-null  float64
 8   Product_Category_3          783667 non-null  float64
 9   Purchase                    550068 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 65.8+ MB


In [4]:
file['Product_Category_2'] = file['Product_Category_2'].apply(np.int64)
file['Product_Category_3'] = file['Product_Category_3'].apply(np.int64)
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 233598
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      783667 non-null  object 
 1   Age                         783667 non-null  object 
 2   Occupation                  783667 non-null  int64  
 3   City_Category               783667 non-null  object 
 4   Stay_In_Current_City_Years  783667 non-null  object 
 5   Marital_Status              783667 non-null  int64  
 6   Product_Category_1          783667 non-null  int64  
 7   Product_Category_2          783667 non-null  int64  
 8   Product_Category_3          783667 non-null  int64  
 9   Purchase                    550068 non-null  float64
dtypes: float64(1), int64(5), object(4)
memory usage: 65.8+ MB


In [5]:
file.replace(['M'], 0, inplace=True)
file.replace(['F'], 1, inplace=True)

In [6]:
file["sum_category"] = file["Product_Category_1"] + file["Product_Category_2"] + file["Product_Category_3"]

In [7]:
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 233598
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      783667 non-null  int64  
 1   Age                         783667 non-null  object 
 2   Occupation                  783667 non-null  int64  
 3   City_Category               783667 non-null  object 
 4   Stay_In_Current_City_Years  783667 non-null  object 
 5   Marital_Status              783667 non-null  int64  
 6   Product_Category_1          783667 non-null  int64  
 7   Product_Category_2          783667 non-null  int64  
 8   Product_Category_3          783667 non-null  int64  
 9   Purchase                    550068 non-null  float64
 10  sum_category                783667 non-null  int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 71.7+ MB


In [8]:
# file.drop(['Product_Category_1', 'Product_Category_2', 'Product_Category_3'], 1, inplace=True)

In [9]:
file = pd.get_dummies(file, columns=['Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years',
                                     'sum_category'])

In [10]:
file

,Gender,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Age_0-17,Age_18-25,Age_26-35,Age_36-45,...,sum_category_36,sum_category_37,sum_category_38,sum_category_39,sum_category_40,sum_category_41,sum_category_42,sum_category_43,sum_category_44,sum_category_48
0,1,0,3,0,0,8370.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,6,14,15200.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,12,0,0,1422.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,12,14,0,1057.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,8,0,0,7969.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233594,1,1,8,0,0,NaN,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
233595,1,1,5,8,0,NaN,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
233596,1,1,1,5,12,NaN,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
233597,1,0,10,16,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
train_data = file[:checkpoint]
test_data = file[checkpoint:]
train_data

,Gender,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Age_0-17,Age_18-25,Age_26-35,Age_36-45,...,sum_category_36,sum_category_37,sum_category_38,sum_category_39,sum_category_40,sum_category_41,sum_category_42,sum_category_43,sum_category_44,sum_category_48
0,1,0,3,0,0,8370.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,6,14,15200.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,12,0,0,1422.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,12,14,0,1057.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,8,0,0,7969.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550063,0,1,20,0,0,368.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550064,1,0,20,0,0,371.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
550065,1,1,20,0,0,137.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
550066,1,0,20,0,0,365.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train_data_X = train_data.loc[:, ~train_data.columns.isin(['Purchase'])].values
train_data_Y = train_data.loc[:, 'Purchase'].values
train_data_X, train_data_Y

(array([[ 1,  0,  3, ...,  0,  0,  0],
        [ 1,  0,  1, ...,  0,  0,  0],
        [ 1,  0, 12, ...,  0,  0,  0],
        ...,
        [ 1,  1, 20, ...,  0,  0,  0],
        [ 1,  0, 20, ...,  0,  0,  0],
        [ 1,  1, 20, ...,  0,  0,  0]], dtype=int64),
 array([ 8370., 15200.,  1422., ...,   137.,   365.,   490.]))

In [13]:
reg_model = linear_model.LinearRegression()
reg_model.fit(train_data_X, train_data_Y)

LinearRegression()

In [14]:
test_data_X = test_data.loc[:, ~train_data.columns.isin(['Purchase'])].values
y_pred = reg_model.predict(test_data_X)
y_pred

array([10278.265625,  9726.203125,  6403.5625  , ..., 11971.875   ,
        8120.5625  ,  8089.46875 ])

In [15]:
submit = test_file.loc[:, test_file.columns.isin(['User_ID', 'Product_ID'])]
submit

,User_ID,Product_ID
0,1000004,P00128942
1,1000009,P00113442
2,1000010,P00288442
3,1000010,P00145342
4,1000011,P00053842
...,...,...
233594,1006036,P00118942
233595,1006036,P00254642
233596,1006036,P00031842
233597,1006037,P00124742


In [16]:
submit.insert(0, "Purchase", y_pred, True)
submit.to_csv(current  + r"\..\rawdata\submit1.csv")